# Kings County Housing Prices Prediction

# Read in
Read in hold out data, scalers, and best model

In [1]:
import pandas as pd
import numpy as np

In [3]:
model = pd.read_pickle('Data/model.pickle')
scaler = pd.read_pickle('Data/scaler.pickle')

In [5]:
df = pd.read_csv('/Users/hasan/Desktop/house-price-pridiction-with-statistical-test/Data/kc_house_data_test_features.csv', index_col=0)

# Feature Engineering
Feature Engineering for holdout set

## Date

In [4]:
df['year'] = pd.to_datetime(df.date).dt.year
df['month'] = pd.to_datetime(df.date).dt.month

In [5]:
condlist = [df.month.isin([4,5,6,7]), df.month.isin([1,2,11,12])]
choicelist = ['summer','winter']
df['season'] = np.select(condlist, choicelist, 'spring&fall')

In [6]:
df = pd.get_dummies(df, columns=['season'], prefix='s', drop_first=True)

## Bedrooms

In [7]:
condlist = [df.bedrooms<2, df.bedrooms>5]
choicelist = [1, 6]
df['bed'] = np.select(condlist, choicelist, df.bedrooms)

In [8]:
df = pd.get_dummies(df, columns=['bed'], drop_first=True)

## Condition

In [9]:
df = pd.get_dummies(df, columns=['condition'], prefix='con', drop_first=True)

## Grade

In [10]:
condlist = [df.grade<6, df.grade>10]
choicelist = [5, 11]
df['rating'] = np.select(condlist, choicelist, df.grade)

In [11]:
df = pd.get_dummies(df, columns=['rating'], prefix='r', drop_first=True)

## Sqft_basement

In [12]:
condlist = [df.sqft_basement==0, df.sqft_basement<1000, df.sqft_basement<2000]
choicelist = [0, 1000, 2000]
df['basement'] = np.select(condlist, choicelist, 3000)

In [13]:
df = pd.get_dummies(df, columns=['basement'], prefix='base', drop_first=True)

## Renovate
not renovated: house_age = yr_sold - yr_built     
renovated: house_age = yr_sold - yr_rebuilt \
yr_rebuilt = yr_renovated * r + yr_built * (1-r)

In [14]:
r = 0.5
df['age'] = np.where(df.yr_renovated==0, df.year-df.yr_built, df.year-df.yr_renovated*r-df.yr_built*(1-r))

## Zipcode

In [15]:
df = pd.get_dummies(df, columns=['zipcode'], prefix='zip', drop_first=True)

## Bathrooms

In [16]:
df['bath'] = df.bathrooms/df.sqft_living

## Waterfront
Interaction between a binary variable (dummy variable) and a continuous variable

In [17]:
df['water'] = df.waterfront*df.sqft_living15

# Prediction
Predict the holdout set

In [18]:
col = ['id','date','bedrooms','bathrooms','floors','waterfront','view','grade','sqft_above','year','month',
       'sqft_basement','yr_built','yr_renovated','lat','long','sqft_living15','sqft_lot', 'sqft_lot15']

In [19]:
dfn = df.drop(columns=col, axis=1)

In [20]:
df_scaled = scaler.transform(dfn)

In [21]:
final = model.predict(df_scaled)

# Export Predictions

In [8]:
final_answers = pd.DataFrame(final)

In [9]:
final_answers.to_csv('Data/housing_preds_Muhammad_Qayyum.csv')